In [6]:
import os, subprocess
import json
import uproot
import awkward as ak
import numpy as np
from coffea import processor, util, hist
import pandas as pd
import pickle

In [7]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.9

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

In [8]:
year = '2017'
nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
outsum = processor.dict_accumulator()

In [9]:
repickle=True

# Check if pickle exists, and don't re-create it if it does
picklename = 'pickles/'+year+'_cutflow.pkl'
if os.path.isfile(picklename):
    repickle=False

In [10]:
# Load all files - this takes a while
if repickle:
    nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
    for n in range(1,nfiles+1):

        with open('infiles-split/'+year+'_'+str(n)+'.json') as f:
            infiles = json.load(f)
    
        filename = '/myeosdir/vh-category/outfiles/'+year+'_'+str(n)+'.coffea'
        #filename = 'outfiles/'+year+'_'+str(n)+'.coffea'
        if os.path.isfile(filename):
            out = util.load(filename)
            outsum.add(out)
        else:
            print('Missing file '+str(n),infiles.keys())
            #print("File " + filename + " is missing")
        
    scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
    outsum['cutflow'].scale(scale_lumi, 'dataset')
    
    # Use pmap to group the datasets together
    cutflow = outsum['cutflow'].group('dataset', hist.Cat('process', 'Process'), pmap)

    outfile = open(picklename, 'wb')
    pickle.dump(cutflow, outfile, protocol=-1)
    outfile.close()

Missing file 2416 dict_keys(['WJetsToLNu_HT-800To1200_TuneCP5_13TeV-madgraphMLM-pythia8'])


/opt/conda/lib/python3.8/site-packages/coffea/hist/hist_tools.py:376: RuntimeWarning: Not all requested indices present in <Cat (name=dataset) instance at 0x7f13c541a490>
  warnings.warn(


In [11]:
# Read the histogram from the pickle file
cutflow = pickle.load(open(picklename,'rb')).sum('genflavor').integrate('region','signal')

In [12]:
cutflow.values()

{('ZH',): array([2.21744089e+04, 2.21744089e+04, 2.21744089e+04, 9.48982371e+01,
        4.03949744e+01, 4.03949744e+01, 3.73694584e+01, 1.36726420e+01,
        5.14016894e+00, 4.77353854e+00, 4.23624421e+00, 2.08523169e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 ('WH',): array([2.30625891e+04, 2.30625891e+04, 2.30625891e+04, 9.13200063e+01,
        3.96660728e+01, 3.96660728e+01, 3.61223798e+01, 1.21142667e+01,
        5.31856405e+00, 4.88749772e+00, 4.65734800e+00, 2.23229621e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 ('ttH',): array([8.62351233e+03, 8.62351233e+03, 8.62351233e+03, 4.38075193e+02,
        1.84986424e+02, 1.84986424e+02, 1.28764881e+02, 1.46393429e+01,
        4.52636545e+00, 3.56481177e+00, 2.34962050e+00, 7.02924380e-01,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 ('VBF',): array([8.95945083e+04, 8.95945083e+04, 8.95945083e+04, 8.99895876e+01,
        2.35016752e+01, 2.35016752e+01, 2.14525623e+01, 4.44142752e+

In [13]:
df1 = pd.DataFrame([])

df1['ggF'] = cutflow.values()[('ggF',)]
df1['VBF'] = cutflow.values()[('VBF',)]
df1['WH'] = cutflow.values()[('WH',)]
df1['ZH'] = cutflow.values()[('ZH',)]
df1['ttH'] = cutflow.values()[('ttH',)]

df1 = df1[3:-3].astype('int')
df1.index = ['Jet 1 kinematics','Jet 2 kinematics','Jet ID','Jet acceptance','Jet $N_2^\text{DDT}$','Opp. hem. b veto','MET $<$ 140 GeV','No leptons','DeepDoubleB']

df1.to_latex(buf=year+'/cutflow-sig.tex')

In [14]:
df2 = pd.DataFrame([])

df2['QCD'] = cutflow.values()[('QCD',)]
df2['Wjets'] = cutflow.values()[('Wjets',)]
df2['Zjets'] = cutflow.values()[('Zjets',)]
df2['VV'] = cutflow.values()[('VV',)]
df2['ttbar'] = cutflow.values()[('ttbar',)]
df2['singlet'] = cutflow.values()[('singlet',)]

df2 = df2[3:-3].astype('int')
df2.index = ['Jet 1 kinematics','Jet 2 kinematics','Jet ID','Jet acceptance','Jet $N_2^\text{DDT}$','Opp. hem. b veto','MET $<$ 140 GeV','No leptons','DeepDoubleB']

df2.to_latex(buf=year+'/cutflow-bkg.tex')